In [1]:
import numpy as np
import pandas as pd

In [2]:
import matplotlib.pyplot as plt
from matplotlib import style
import matplotlib.ticker as ticker
import seaborn as sns

In [3]:
from sklearn.datasets import load_boston
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ParameterGrid
from sklearn.inspection import permutation_importance
import multiprocessing

In [4]:
labels = pd.read_csv('../csv/train_labels.csv')
labels.head()

,building_id,damage_grade
0,802906,3
1,28830,2
2,94947,3
3,590882,2
4,201944,3


In [5]:
values = pd.read_csv('../csv/train_values.csv')
values.head(10).T

,0,1,2,3,4,5,6,7,8,9
building_id,802906,28830,94947,590882,201944,333020,728451,475515,441126,989500
geo_level_1_id,6,8,21,22,11,8,9,20,0,26
geo_level_2_id,487,900,363,418,131,558,475,323,757,886
geo_level_3_id,12198,2812,8973,10694,1488,6089,12066,12236,7219,994
count_floors_pre_eq,2,2,2,2,3,2,2,2,2,1
age,30,10,10,10,30,10,25,0,15,0
area_percentage,6,8,5,6,8,9,3,8,8,13
height_percentage,5,7,5,5,9,5,4,6,6,4
land_surface_condition,t,o,t,t,t,t,n,t,t,t
foundation_type,r,r,r,r,r,r,r,w,r,i


In [6]:
to_be_categorized = ["land_surface_condition", "foundation_type", "roof_type",\
                     "position", "ground_floor_type", "other_floor_type",\
                     "plan_configuration", "legal_ownership_status"]
for row in to_be_categorized:
    values[row] = values[row].astype("category")
values.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260601 entries, 0 to 260600
Data columns (total 39 columns):
 #   Column                                  Non-Null Count   Dtype   
---  ------                                  --------------   -----   
 0   building_id                             260601 non-null  int64   
 1   geo_level_1_id                          260601 non-null  int64   
 2   geo_level_2_id                          260601 non-null  int64   
 3   geo_level_3_id                          260601 non-null  int64   
 4   count_floors_pre_eq                     260601 non-null  int64   
 5   age                                     260601 non-null  int64   
 6   area_percentage                         260601 non-null  int64   
 7   height_percentage                       260601 non-null  int64   
 8   land_surface_condition                  260601 non-null  category
 9   foundation_type                         260601 non-null  category
 10  roof_type                       

In [7]:
datatypes = dict(values.dtypes)
for row in values.columns:
    if datatypes[row] != "int64" and datatypes[row] != "int32" and \
       datatypes[row] != "int16" and datatypes[row] != "int8":
        continue
    if values[row].nlargest(1).item() > 32767 and values[row].nlargest(1).item() < 2**31:
        values[row] = values[row].astype(np.int32)
    elif values[row].nlargest(1).item() > 127:
        values[row] = values[row].astype(np.int16)
    else:
        values[row] = values[row].astype(np.int8)
values.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260601 entries, 0 to 260600
Data columns (total 39 columns):
 #   Column                                  Non-Null Count   Dtype   
---  ------                                  --------------   -----   
 0   building_id                             260601 non-null  int32   
 1   geo_level_1_id                          260601 non-null  int8    
 2   geo_level_2_id                          260601 non-null  int16   
 3   geo_level_3_id                          260601 non-null  int16   
 4   count_floors_pre_eq                     260601 non-null  int8    
 5   age                                     260601 non-null  int16   
 6   area_percentage                         260601 non-null  int8    
 7   height_percentage                       260601 non-null  int8    
 8   land_surface_condition                  260601 non-null  category
 9   foundation_type                         260601 non-null  category
 10  roof_type                       

In [8]:
labels["building_id"] = labels["building_id"].astype(np.int32)
labels["damage_grade"] = labels["damage_grade"].astype(np.int8)
labels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260601 entries, 0 to 260600
Data columns (total 2 columns):
 #   Column        Non-Null Count   Dtype
---  ------        --------------   -----
 0   building_id   260601 non-null  int32
 1   damage_grade  260601 non-null  int8 
dtypes: int32(1), int8(1)
memory usage: 1.2 MB


In [9]:
importantes = values\
                .merge(labels, on="building_id")
importantes.drop(columns=["building_id"], inplace = True)
importantes["geo_level_1_id"] = importantes["geo_level_1_id"].astype("category")

In [10]:
X_train, X_test, y_train, y_test = train_test_split(importantes.drop(columns = 'damage_grade'),
                                                    importantes['damage_grade'],\
                                                    test_size = 0.2, random_state = 123)

In [11]:
def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
    res = pd.concat([original_dataframe, dummies], axis=1)
    res = res.drop([feature_to_encode], axis=1)
    return(res) 

In [12]:
features = ["geo_level_1_id", "land_surface_condition",\
                      "foundation_type", "roof_type",\
                     "position", "ground_floor_type", "other_floor_type",\
                     "plan_configuration", "legal_ownership_status"]

In [13]:
for feature in features:
    X_train = encode_and_bind(X_train, feature)

In [14]:
modelito = RandomForestClassifier(n_estimators = 270, max_depth = 25, criterion = "gini", verbose=True, n_jobs=-1)
modelito.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    9.4s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   43.3s
[Parallel(n_jobs=-1)]: Done 270 out of 270 | elapsed:   59.7s finished


RandomForestClassifier(max_depth=25, n_estimators=270, n_jobs=-1, verbose=True)

In [15]:
modelito.score(X_train, y_train)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    1.3s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    5.7s
[Parallel(n_jobs=4)]: Done 270 out of 270 | elapsed:    8.0s finished


0.8170184190330008

In [16]:
modelito.feature_importances_

array([7.82854500e-02, 8.03168593e-02, 1.86600493e-02, 6.77002129e-02,
       6.93451238e-02, 4.15999865e-02, 6.43265895e-03, 2.26278562e-02,
       5.93449843e-03, 3.97626056e-03, 6.66715045e-03, 1.41121032e-02,
       1.25327296e-02, 7.07811716e-03, 5.89349996e-03, 4.83722743e-03,
       3.66995067e-03, 1.89034526e-02, 8.39109200e-03, 5.20249747e-03,
       4.49225583e-03, 1.79354149e-03, 3.39809451e-04, 1.55589890e-04,
       4.32824981e-04, 7.41148853e-05, 4.68927285e-05, 2.69048953e-05,
       1.19002840e-03, 3.20312169e-03, 2.33870624e-03, 3.38425286e-04,
       2.35222105e-03, 1.09855032e-02, 2.91538834e-03, 8.99064769e-03,
       5.47737117e-03, 1.72298143e-02, 2.16826558e-03, 7.32089212e-03,
       3.15302559e-03, 9.30980861e-04, 9.43907349e-03, 8.34807336e-04,
       5.51809055e-04, 2.13636964e-03, 1.19720220e-01, 1.28252160e-02,
       1.11892104e-04, 9.41695701e-03, 2.04546723e-02, 5.31543394e-03,
       2.52355038e-04, 1.05151029e-03, 5.02846791e-03, 2.15288860e-02,
      

In [17]:
importantes = values\
                .merge(labels, on="building_id")
importantes.drop(columns=["building_id"], inplace = True)
importantes["geo_level_1_id"] = importantes["geo_level_1_id"].astype("category")
importantes

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade
0,6,487,12198,2,30,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,3
1,8,900,2812,2,10,8,7,o,r,n,...,0,0,0,0,0,0,0,0,0,2
2,21,363,8973,2,10,5,5,t,r,n,...,0,0,0,0,0,0,0,0,0,3
3,22,418,10694,2,10,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,2
4,11,131,1488,3,30,8,9,t,r,n,...,0,0,0,0,0,0,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260596,25,1335,1621,1,55,6,3,n,r,n,...,0,0,0,0,0,0,0,0,0,2
260597,17,715,2060,2,0,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,3
260598,17,51,8163,3,55,6,7,t,r,q,...,0,0,0,0,0,0,0,0,0,3
260599,26,39,1851,2,10,14,6,t,r,x,...,0,0,0,0,0,0,0,0,0,2


In [18]:
X_train, X_test, y_train, y_test = train_test_split(importantes.drop(columns = 'damage_grade'),
                                                    importantes['damage_grade'],\
                                                    test_size = 0.2, random_state = 123)

In [19]:
def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
    res = pd.concat([original_dataframe, dummies], axis=1)
    res = res.drop([feature_to_encode], axis=1)
    return(res) 

In [20]:
features = ["geo_level_1_id", "land_surface_condition",\
                      "foundation_type", "roof_type",\
                     "position", "ground_floor_type", "other_floor_type",\
                     "plan_configuration", "legal_ownership_status"]

In [21]:
for feature in features:
    X_train = encode_and_bind(X_train, feature)
    X_test = encode_and_bind(X_test, feature)

In [22]:
modelito = RandomForestClassifier(n_estimators = 270, max_depth = 25, max_features = 45, criterion = "gini", verbose=True, n_jobs=-1)
modelito.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   30.8s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 270 out of 270 | elapsed:  3.1min finished


RandomForestClassifier(max_depth=25, max_features=45, n_estimators=270,
                       n_jobs=-1, verbose=True)

In [23]:
modelito.score(X_train, y_train)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    1.6s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    6.7s
[Parallel(n_jobs=4)]: Done 270 out of 270 | elapsed:    9.3s finished


0.8879556792018419

In [24]:
y_preds = modelito.predict(X_test)
f1_score(y_test, y_preds, average='micro')

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.4s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    1.7s
[Parallel(n_jobs=4)]: Done 270 out of 270 | elapsed:    2.4s finished


0.7323727480286257

In [25]:
modelito.feature_importances_

array([1.16499353e-01, 1.13556459e-01, 1.69074697e-02, 8.38433669e-02,
       8.53906263e-02, 4.75613910e-02, 5.45145021e-03, 1.65444762e-02,
       5.32912413e-03, 2.78424762e-03, 5.50610293e-03, 7.95090417e-03,
       1.20597617e-02, 5.47937459e-03, 3.71932697e-03, 1.75037715e-03,
       3.14665956e-03, 2.14986269e-02, 7.90213916e-03, 4.90870144e-03,
       3.54308339e-03, 1.35927039e-03, 2.57407423e-04, 1.03177374e-04,
       3.80567136e-04, 5.19305312e-05, 2.83351718e-05, 2.38600670e-05,
       1.08257503e-03, 3.53618560e-04, 5.23007366e-04, 3.64616954e-04,
       3.64340651e-03, 2.60942592e-03, 7.03948826e-04, 5.56024750e-03,
       7.13812743e-03, 1.96501646e-02, 1.35551841e-03, 1.04089734e-02,
       5.25848410e-03, 3.17129789e-04, 3.47519627e-03, 1.75870542e-04,
       2.22095887e-04, 4.58008265e-04, 9.67480307e-02, 1.30745696e-02,
       4.45370921e-05, 2.07095644e-03, 2.15983511e-02, 8.34089014e-04,
       2.48447942e-04, 4.37687754e-04, 8.02884421e-04, 1.38316732e-02,
      

In [26]:
test_values = pd.read_csv('../csv/test_values.csv', index_col = "building_id")
test_values

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
building_id,,,,,,,,,,,,,,,,,,,,,
300051,17,596,11307,3,20,7,6,t,r,n,...,0,0,0,0,0,0,0,0,0,0
99355,6,141,11987,2,25,13,5,t,r,n,...,1,0,0,0,0,0,0,0,0,0
890251,22,19,10044,2,5,4,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
745817,26,39,633,1,0,19,3,t,r,x,...,0,0,1,0,0,0,0,0,0,0
421793,17,289,7970,3,15,8,7,t,r,q,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310028,4,605,3623,3,70,20,6,t,r,q,...,1,0,0,0,0,0,0,0,0,0
663567,10,1407,11907,3,25,6,7,n,r,n,...,0,0,0,0,0,0,0,0,0,0
1049160,22,1136,7712,1,50,3,3,t,r,n,...,0,0,0,0,0,0,0,0,0,0


In [27]:
subset_de_testvalues = test_values
subset_de_testvalues["geo_level_1_id"] = subset_de_testvalues["geo_level_1_id"].astype("category")
subset_de_testvalues

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
building_id,,,,,,,,,,,,,,,,,,,,,
300051,17,596,11307,3,20,7,6,t,r,n,...,0,0,0,0,0,0,0,0,0,0
99355,6,141,11987,2,25,13,5,t,r,n,...,1,0,0,0,0,0,0,0,0,0
890251,22,19,10044,2,5,4,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
745817,26,39,633,1,0,19,3,t,r,x,...,0,0,1,0,0,0,0,0,0,0
421793,17,289,7970,3,15,8,7,t,r,q,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310028,4,605,3623,3,70,20,6,t,r,q,...,1,0,0,0,0,0,0,0,0,0
663567,10,1407,11907,3,25,6,7,n,r,n,...,0,0,0,0,0,0,0,0,0,0
1049160,22,1136,7712,1,50,3,3,t,r,n,...,0,0,0,0,0,0,0,0,0,0


In [28]:
def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
    res = pd.concat([original_dataframe, dummies], axis=1)
    res = res.drop([feature_to_encode], axis=1)
    return(res) 

In [29]:
features = ["geo_level_1_id", "land_surface_condition",\
            "foundation_type", "roof_type",\
                     "position", "ground_floor_type", "other_floor_type",\
                     "plan_configuration", "legal_ownership_status"]

In [30]:
for feature in features:
    subset_de_testvalues = encode_and_bind(subset_de_testvalues, feature)
    subset_de_testvalues

In [31]:
preds = modelito.predict(subset_de_testvalues)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.6s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    2.7s
[Parallel(n_jobs=4)]: Done 270 out of 270 | elapsed:    3.8s finished


In [32]:
submission_format = pd.read_csv('../csv/submission_format.csv', index_col = "building_id")

In [33]:
my_submission = pd.DataFrame(data=preds,
                             columns=submission_format.columns,
                             index=submission_format.index)

In [34]:
my_submission.head()

,damage_grade
building_id,
300051,3
99355,2
890251,2
745817,1
421793,3


In [35]:
my_submission.to_csv('../csv/predictions/laucha1.csv')